In [1]:
"""
www.kyero.com web scrapper
Date created: 2018 07 12
Prepared by Vytautas Bielinskas at Ekistics Real Estate Advisors

"""

'\nwww.kyero.com web scrapper\nDate created: 2018 07 12\nPrepared by Vytautas Bielinskas at Ekistics Real Estate Advisors\n\n'

In [2]:
# ::: ------[Import libraries]------ ::: #
import numpy as np
import pandas as pd
import os, datetime, requests, time
from bs4 import BeautifulSoup
from selenium import webdriver

options = webdriver.chrome.options.Options()
options.add_argument("--disable-extensions")

In [3]:
# ::: ------[Set primary URL and Selenium]------ ::: #
base_url = 'https://www.kyero.com/en/malaga-property-for-sale-0l31661?&new_build=true&page=1'
options = webdriver.chrome.options.Options()

# ::: ------[Disable Pop-ups!]------ ::: #
chrome_prefs = {}
option = webdriver.ChromeOptions()
option.experimental_options["prefs"] = chrome_prefs
chrome_prefs["profile.default_content_settings"] = { "popups": 1 }

options.add_argument("--disable-extensions")
    
chrome_path = r"E:\python\10 RealWorldExamples\webscrapping\Booking.Com\chromedriver.exe"
driver = webdriver.Chrome(chrome_path)
driver.get(base_url)

# Window for internal pages
new_window = webdriver.Chrome(chrome_path)

In [4]:
def isLastPage(url):
    last_page = False
    
    try:
        driver.find_element_by_class_name('no-results__message')
        last_page = True
        print('{}\nThat is all. No properties more.\n{}'.format('-' *40, '-' * 40))
    except:
        last_page = False
    return last_page

In [5]:
# ::: ------[Generate URL list for scrapping]------ ::: #
def getURLs(n = 999):
    urls = []
    base_url = 'https://www.kyero.com/en/malaga-property-for-sale-0l31661?&new_build=true&page='
    
    for i in range(1, n, 1):
        urls.append(base_url + str(i))
    return urls

urls = getURLs()

In [6]:
line = '-' * 30

In [7]:
datalist = []      # <<< Here our data will be storage
page_index = 0
property_index = 0

def getDataInside(property_url, new_window):
    features = {}

    r = requests.get(property_url)
    c = r.content
    html = BeautifulSoup(c, "html.parser")
    
    table = html.find('table', {'class':'table-features'})
    
    for row in table.findAll("tr"):
        cells_html = row.findAll("td")
        
        cell_name = cells_html[0].text
        cell_value = cells_html[1].text
        
        if 'new build'.upper() not in cell_name.upper():
            #print('{} : {}'.format(cell_name, cell_value))
            
            features['*' + cell_name.upper() + '*'] = cell_value
            
    new_window.get(property_url)
    time.sleep(2)
    
    gmaps = new_window.find_element_by_class_name('property__map').get_attribute('innerHTML')
    
    lat = gmaps.split('"lat":')[1].split(',')[0]
    lon = gmaps.split('"lon":')[1].split('}')[0]
    
    features['LAT'] = lat
    features['LON'] = lon
    
    print("GEO: {}, {}".format(lat, lon))
            
    #print('TABLE INSIDE:\n{}.'.format(table))
    
    print(features)
    return features
# -------------------------------------------------------------------------------

while not isLastPage(urls[page_index]):
    
    print('PAGE NUMBER: {}, -  {}'.format(page_index, urls[page_index]))
    
    # ::: ------[Get all properties in a page]------ ::: #
    all_properties_page = driver.find_elements_by_class_name('card-property')
    print('We found {} properties in this page.'.format(len(all_properties_page)))

    for element in range(0, len(all_properties_page), 1):
        property_index = property_index + 1
        print('Property: {}.'.format(property_index))

        record = {}

        this_property = all_properties_page[element].get_attribute('innerHTML')

        title = this_property.split('<img alt="')[1].split('" src="')[0]
        print('TITLE: {}'.format(title))

        img = this_property.split('src="')[1].split('?')[0].split('q82/')[1]
        print('IMG: {}'.format(img))

        price = float(this_property.split('€ <!-- /react-text --><span>')[1].split('</span')[0].replace(',', ''))
        print('PRICE: {}'.format(price))

        inline_features = all_properties_page[element].find_element_by_class_name('card-property__features').get_attribute('innerHTML')
        #print('_Inline features:\n{}'.format(inline_features))

        bedrooms = inline_features.split('</span><span><i class="kicon el-ml-sm el-mr-sm kicon-bed">')[0].split('>')[2]
        
        try:
            bathrooms = inline_features.split('</span><span><i class="kicon el-ml-sm el-mr-sm kicon-bath">')[0].split('>')[10]
            if 'm²'.upper() in str(bathrooms).upper() or 'span'.upper() in str(bathrooms).upper():
                bathrooms = None
        except:
            print('(!!!) Something wrong with BATHROOM (!!!)')
            bathrooms = None
            
        size = inline_features.split(' m²')[0].split('>')[-1]

        print('\nBEDROOMS: {}'.format(bedrooms))
        print('BATHROOMS: {}'.format(bathrooms))
        print('SIZE: {}'.format(size))

        # Checking if the property is New
        new_build = False
        if 'new build'.upper() in this_property.upper():
            new_build = True
        else:
            new_build = False

        print('NEW BUILD: {}'.format(new_build))

        # Checking if the property has a Pool
        pool = False
        if 'pool'.upper() in this_property.upper():
            pool = True
        else:
            pool = False

        print('POOL: {}'.format(pool))

        # Determine property ID
        this_id = img.split('/')[-2]
        print('ID: {}'.format(this_id))

        # Generating property URL
        this_url = 'https://www.kyero.com/en/property/' + str(this_id)
        print('URL: {}'.format(this_url))

        # Getting data from internal page
        record = getDataInside(this_url, new_window)

        # ::: ------[Write primary data to the dictionary]------ ::: #
        record['TITLE'] = title
        record['IMG'] = img
        record['PRICE'] = price
        record['BEDROOMS'] = bedrooms
        record['BATHROOMS'] = bathrooms
        record['SIZE'] = size
        record['NEW BUILD'] = new_build
        record['POOL'] = pool
        record['ID'] = this_id
        record['URL'] = this_url
        try:
            record['PRICE PER SQ. M.'] = float(price) / float(size)
        except:
            record['PRICE PER SQ. M.'] = None

        datalist.append(dict(record))

        print(line)
        
    # ::: ------[Open new search page]------ ::: #
    page_index = page_index + 1
    driver.get(urls[page_index])
    time.sleep(5)


PAGE NUMBER: 0, -  https://www.kyero.com/en/malaga-property-for-sale-0l31661?&new_build=true&page=1
We found 20 properties in this page.
Property: 1.
TITLE: 2 bedroom Apartment for sale in Malaga
IMG: http://production.kyero.s3.amazonaws.com/5460/5460821/72890267_original.jpg
PRICE: 365000.0

BEDROOMS: 2
BATHROOMS: 2
SIZE: 77
NEW BUILD: True
POOL: True
ID: 5460821
URL: https://www.kyero.com/en/property/5460821
GEO: 36.719623, -4.420023
{'*POOL*': 'communal pool', '*UTILITIES*': 'adsl', '*OTHER*': 'lift, marble floors, satellite tv, fitted wardrobes, double glazing', '*SECURITY*': 'gated complex', 'LAT': '36.719623', '*HEATING*': 'air conditioning', '*ROOMS*': 'utility room, store room', '*KITCHEN*': 'fitted kitchen', '*PARKING*': 'underground parking', '*QUALITY*': 'immaculate condition, newly built', '*LOCATION*': 'near transport, close to town, close to the sea, close to shops, close to port, close to golf, first line beach, commercial area, close to schools', '*BATHROOM*': 'en suite

In [21]:
df = pd.DataFrame(datalist)

In [22]:
df.columns

Index(['*BATHROOM*', '*BUILDINGS*', '*EMISSIONS RATING*',
       '*ENERGY CONSUMPTION RATING*', '*GARDEN*', '*HEATING*', '*KITCHEN*',
       '*LEISURE*', '*LOCATION*', '*OTHER*', '*OUTSIDE*', '*PARKING*',
       '*PLOT SIZE*', '*POOL*', '*QUALITY*', '*ROOMS*', '*SECURITY*',
       '*TENURE*', '*UTILITIES*', '*VIEWS*', 'BATHROOMS', 'BEDROOMS', 'ID',
       'IMG', 'LAT', 'LON', 'NEW BUILD', 'POOL', 'PRICE', 'PRICE PER SQ. M.',
       'SIZE', 'TITLE', 'URL'],
      dtype='object')

In [27]:
df.head(10)

,ID,TITLE,PRICE,PRICE PER SQ. M.,BEDROOMS,BATHROOMS,SIZE,POOL,URL,IMG,...,*UTILITIES*,*ENERGY CONSUMPTION RATING*,*SECURITY*,*PARKING*,*PLOT SIZE*,*LEISURE*,*VIEWS*,*QUALITY*,LAT,LON
0,5460821,2 bedroom Apartment for sale in Malaga,365000.0,4740.259740,2,2,77,True,https://www.kyero.com/en/property/5460821,http://production.kyero.s3.amazonaws.com/5460/...,...,adsl,Awaiting details,gated complex,underground parking,NaN,NaN,NaN,"immaculate condition, newly built",36.719623,-4.420023
1,5460803,4 bedroom Apartment for sale in Malaga,339000.0,2511.111111,4,2,135,True,https://www.kyero.com/en/property/5460803,http://production.kyero.s3.amazonaws.com/5460/...,...,adsl,Awaiting details,gated complex,underground parking,NaN,NaN,mountain views,"immaculate condition, newly built, new develop...",36.719623,-4.420023
2,5460820,2 bedroom Apartment for sale in Malaga,309000.0,3551.724138,2,2,87,True,https://www.kyero.com/en/property/5460820,http://production.kyero.s3.amazonaws.com/5460/...,...,adsl,Awaiting details,gated complex,underground parking,NaN,NaN,NaN,"immaculate condition, newly built",36.719623,-4.420023
3,5363194,2 bedroom Apartment for sale in Malaga,266000.0,3546.666667,2,1,75,True,https://www.kyero.com/en/property/5363194,http://production.kyero.s3.amazonaws.com/5363/...,...,"electricity, adsl",Awaiting details,electronic entry system,parking,NaN,NaN,NaN,"immaculate condition, newly built",36.719623,-4.420023
4,4438626,3 bedroom Apartment for sale in Malaga,1090000.0,6264.367816,3,2,174,False,https://www.kyero.com/en/property/4438626,http://production.kyero.s3.amazonaws.com/4438/...,...,NaN,Awaiting details,NaN,NaN,NaN,NaN,NaN,"immaculate condition, well presented, tasteful...",36.719623,-4.420023
5,5159693,2 bedroom Apartment for sale in Malaga,266000.0,3546.666667,2,1,75,True,https://www.kyero.com/en/property/5159693,http://production.kyero.s3.amazonaws.com/5159/...,...,"electricity, adsl",Awaiting details,electronic entry system,parking,NaN,NaN,NaN,"immaculate condition, newly built",36.719623,-4.420023
6,5308467,4 bedroom Apartment for sale in Malaga,402000.0,2284.090909,4,3,176,True,https://www.kyero.com/en/property/5308467,http://production.kyero.s3.amazonaws.com/5308/...,...,NaN,Awaiting details,gated complex,underground parking,NaN,gymnasium,NaN,"immaculate condition, newly built, new develop...",36.719623,-4.420023
7,5437805,3 bedroom Apartment for sale in Malaga,235400.0,2615.555556,3,3,90,True,https://www.kyero.com/en/property/5437805,http://production.kyero.s3.amazonaws.com/5437/...,...,NaN,Awaiting details,NaN,NaN,NaN,NaN,NaN,NaN,36.719623,-4.420023
8,4427021,2 bedroom Apartment for sale in Malaga,309000.0,3121.212121,2,1,99,False,https://www.kyero.com/en/property/4427021,http://production.kyero.s3.amazonaws.com/4427/...,...,NaN,Awaiting details,NaN,NaN,NaN,NaN,NaN,NaN,36.719623,-4.420023
9,5343631,18 bedroom Land for sale in Malaga,250000.0,NaN,18,18,,False,https://www.kyero.com/en/property/5343631,http://production.kyero.s3.amazonaws.com/5343/...,...,NaN,Awaiting details,NaN,NaN,NaN,NaN,NaN,NaN,36.719623,-4.420023


In [23]:
# :::------[Restructuring DataFrame]------::: #
df = df[['ID', 'TITLE', 'PRICE', 'PRICE PER SQ. M.', 'BEDROOMS', 'BATHROOMS', 'SIZE', 'POOL', 
         'URL', 'IMG', 'NEW BUILD','*ROOMS*', '*KITCHEN*', '*BATHROOM*', '*GARDEN*', '*OUTSIDE*', 
         '*HEATING*', '*POOL*', '*BUILDINGS*', '*TENURE*', '*UTILITIES*', '*ENERGY CONSUMPTION RATING*', 
         '*SECURITY*', '*PARKING*', '*PLOT SIZE*', '*LEISURE*', '*VIEWS*', '*QUALITY*', 'LAT', 'LON' 
        ]]

In [24]:
df.columns

Index(['ID', 'TITLE', 'PRICE', 'PRICE PER SQ. M.', 'BEDROOMS', 'BATHROOMS',
       'SIZE', 'POOL', 'URL', 'IMG', 'NEW BUILD', '*ROOMS*', '*KITCHEN*',
       '*BATHROOM*', '*GARDEN*', '*OUTSIDE*', '*HEATING*', '*POOL*',
       '*BUILDINGS*', '*TENURE*', '*UTILITIES*', '*ENERGY CONSUMPTION RATING*',
       '*SECURITY*', '*PARKING*', '*PLOT SIZE*', '*LEISURE*', '*VIEWS*',
       '*QUALITY*', 'LAT', 'LON'],
      dtype='object')

In [28]:
df.head(10)

,ID,TITLE,PRICE,PRICE PER SQ. M.,BEDROOMS,BATHROOMS,SIZE,POOL,URL,IMG,...,*UTILITIES*,*ENERGY CONSUMPTION RATING*,*SECURITY*,*PARKING*,*PLOT SIZE*,*LEISURE*,*VIEWS*,*QUALITY*,LAT,LON
0,5460821,2 bedroom Apartment for sale in Malaga,365000.0,4740.259740,2,2,77,True,https://www.kyero.com/en/property/5460821,http://production.kyero.s3.amazonaws.com/5460/...,...,adsl,Awaiting details,gated complex,underground parking,NaN,NaN,NaN,"immaculate condition, newly built",36.719623,-4.420023
1,5460803,4 bedroom Apartment for sale in Malaga,339000.0,2511.111111,4,2,135,True,https://www.kyero.com/en/property/5460803,http://production.kyero.s3.amazonaws.com/5460/...,...,adsl,Awaiting details,gated complex,underground parking,NaN,NaN,mountain views,"immaculate condition, newly built, new develop...",36.719623,-4.420023
2,5460820,2 bedroom Apartment for sale in Malaga,309000.0,3551.724138,2,2,87,True,https://www.kyero.com/en/property/5460820,http://production.kyero.s3.amazonaws.com/5460/...,...,adsl,Awaiting details,gated complex,underground parking,NaN,NaN,NaN,"immaculate condition, newly built",36.719623,-4.420023
3,5363194,2 bedroom Apartment for sale in Malaga,266000.0,3546.666667,2,1,75,True,https://www.kyero.com/en/property/5363194,http://production.kyero.s3.amazonaws.com/5363/...,...,"electricity, adsl",Awaiting details,electronic entry system,parking,NaN,NaN,NaN,"immaculate condition, newly built",36.719623,-4.420023
4,4438626,3 bedroom Apartment for sale in Malaga,1090000.0,6264.367816,3,2,174,False,https://www.kyero.com/en/property/4438626,http://production.kyero.s3.amazonaws.com/4438/...,...,NaN,Awaiting details,NaN,NaN,NaN,NaN,NaN,"immaculate condition, well presented, tasteful...",36.719623,-4.420023
5,5159693,2 bedroom Apartment for sale in Malaga,266000.0,3546.666667,2,1,75,True,https://www.kyero.com/en/property/5159693,http://production.kyero.s3.amazonaws.com/5159/...,...,"electricity, adsl",Awaiting details,electronic entry system,parking,NaN,NaN,NaN,"immaculate condition, newly built",36.719623,-4.420023
6,5308467,4 bedroom Apartment for sale in Malaga,402000.0,2284.090909,4,3,176,True,https://www.kyero.com/en/property/5308467,http://production.kyero.s3.amazonaws.com/5308/...,...,NaN,Awaiting details,gated complex,underground parking,NaN,gymnasium,NaN,"immaculate condition, newly built, new develop...",36.719623,-4.420023
7,5437805,3 bedroom Apartment for sale in Malaga,235400.0,2615.555556,3,3,90,True,https://www.kyero.com/en/property/5437805,http://production.kyero.s3.amazonaws.com/5437/...,...,NaN,Awaiting details,NaN,NaN,NaN,NaN,NaN,NaN,36.719623,-4.420023
8,4427021,2 bedroom Apartment for sale in Malaga,309000.0,3121.212121,2,1,99,False,https://www.kyero.com/en/property/4427021,http://production.kyero.s3.amazonaws.com/4427/...,...,NaN,Awaiting details,NaN,NaN,NaN,NaN,NaN,NaN,36.719623,-4.420023
9,5343631,18 bedroom Land for sale in Malaga,250000.0,NaN,18,18,,False,https://www.kyero.com/en/property/5343631,http://production.kyero.s3.amazonaws.com/5343/...,...,NaN,Awaiting details,NaN,NaN,NaN,NaN,NaN,NaN,36.719623,-4.420023


In [26]:
df.to_csv('20180713_kyero.com_malaga_scrapped_data_VB_2.csv')